<a href="https://colab.research.google.com/github/AmaleshV/Elliptic/blob/master/elliptic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Increase RAM

In [0]:
#not mandatory if and only to increase the ram from 12GB to 26GB
d=[]
while(1):
  d.append('1')

In [0]:
#reset variables if you've any predefined variables
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Mount google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# Git-lfs install

In [0]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


# Git clone elliptic

In [0]:
%%time
#as the file is >100MB we need to use git lfs rather git clone
!git lfs clone https://github.com/AmaleshV/Elliptic.git

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'Elliptic'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 26 (delta 6), reused 19 (delta 4), pack-reused 0
Unpacking objects: 100% (26/26), done.
Git LFS: (0 of 1 files) 0 B / 146.12 MB
batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.
error: failed to fetch some objects from 'https://github.com/AmaleshV/Elliptic.git/info/lfs'
CPU times: user 18.3 ms, sys: 24.9 ms, total: 43.2 ms
Wall time: 1.39 s


In [0]:
#!rm -rf Elliptic/

In [0]:
#unzip the elliptic dataset
!unzip -uq "/content/Elliptic/elliptic-data-set.zip" -d "/content/"

[/content/Elliptic/elliptic-data-set.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/Elliptic/elliptic-data-set.zip or
        /content/Elliptic/elliptic-data-set.zip.zip, and cannot find /content/Elliptic/elliptic-data-set.zip.ZIP, period.


In [0]:
from os import chdir, getcwd
chdir('/content/drive/My Drive/elliptic')
print(getcwd())

/content/drive/My Drive/elliptic


# From Drive

In [0]:
#as the bandwidth of github lfs is over we've to stick to an alternative
!pip install gdown
!gdown --id 1GE6WIhTkFpzWym1wdtYUyT4AolAtYyhu
!unzip elliptic-data-set.zip

Downloading...
From: https://drive.google.com/uc?id=1GE6WIhTkFpzWym1wdtYUyT4AolAtYyhu
To: /content/elliptic-data-set.zip
153MB [00:04, 30.8MB/s]
Archive:  elliptic-data-set.zip
  inflating: elliptic_bitcoin_dataset/elliptic_txs_classes.csv  
  inflating: elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv  
  inflating: elliptic_bitcoin_dataset/elliptic_txs_features.csv  


# Install required packages

In [0]:
!pip install node2vec
from tkinter.filedialog import askopenfilename
import matplotlib.pyplot as plt
import networkx as nx
from node2vec import Node2Vec
import node2vec
import numpy as np
import numpy
from sklearn.metrics import roc_curve,roc_auc_score,f1_score,cohen_kappa_score,precision_score,recall_score,confusion_matrix
from sklearn.model_selection import ParameterGrid, ParameterSampler
from time import time
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import itemfreq
import plotly.offline as py 
import plotly.graph_objs as go

In [0]:
?Node2Vec.fit

In [0]:
!pip install eif

     |████████████████████████████████| 1.6MB 2.7MB/s 
  Created wheel for eif: filename=eif-2.0.2-cp36-cp36m-linux_x86_64.whl size=250752 sha256=fee8364485c649ec7e06326ff07380ce802ee77da58a86328667135a7f95b308
  Stored in directory: /root/.cache/pip/wheels/24/ff/c3/6a610ba4a6c8e968483ba15a0e38feacd7c0a26c932b2f91bd
Successfully built eif


# Read the csv files

In [0]:
%%time
edges = pd.read_csv('/content/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv',engine='python')
features = pd.read_csv('/content/elliptic_bitcoin_dataset/elliptic_txs_features.csv',engine='python',header=None)
classes = pd.read_csv('/content/elliptic_bitcoin_dataset/elliptic_txs_classes.csv',engine='python')

CPU times: user 35 s, sys: 3.36 s, total: 38.4 s
Wall time: 38.5 s


# Building the graph

In [0]:
len(edges),len(features),len(classes)
display(edges.head(5),features.head(5),classes.head(5))

#defining features
tx_features = ["tx_feat_"+str(i) for i in range(2,95)]
agg_features = ["agg_feat_"+str(i) for i in range(1,73)]
features.columns = ["txId","time_step"] + tx_features + agg_features
features = pd.merge(features,classes,left_on="txId",right_on="txId",how='left')
features['class'] = features['class'].apply(lambda x: '0' if x == "unknown" else x)

#class counts
features.groupby('class').size()
count_by_class = features[["time_step",'class']].groupby(['time_step','class']).size().to_frame().reset_index()

#categorising the class
illicit_count = count_by_class[count_by_class['class'] == '1']
licit_count = count_by_class[count_by_class['class'] == '2']
unknown_count = count_by_class[count_by_class['class'] == "0"]

#Building graph
ids = features['txId']
short_edges = edges[edges['txId1'].isin(ids)]
graph = nx.from_pandas_edgelist(short_edges, source = 'txId1', target = 'txId2', 
                                 create_using = nx.DiGraph())

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,-0.049707,-0.164402,-0.028741,-0.035391,-0.042955,-0.013282,-0.057195,-0.169609,-0.171154,-0.174473,-1.373657,-1.371460,-0.139731,-0.148912,-0.080147,-0.155661,-0.010763,-0.012107,-0.139733,-0.148907,-0.080147,-0.155661,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.127462,0.003143,0.002426,-0.120950,-0.199145,-0.187993,-0.212948,1.064205,1.063787,-1.373782,-1.354735,-0.297975,-1.403698,1.342003,1.340733,-0.171601,-0.458162,-0.423588,-0.440883,-1.015963,-1.016230,-0.968903,-0.375715,0.759748,-0.768329,1.488113,1.487932,-0.216814,-0.605631,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,-0.049707,-0.164417,-0.028741,-0.035391,-0.042955,-0.013282,-0.055327,-0.169757,-0.171477,-0.174490,0.887058,0.884557,-0.139731,-0.148912,-0.080147,-0.155661,-0.010763,-0.012107,-0.139733,-0.148907,-0.080147,-0.155661,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.127462,0.003143,0.002426,-0.121330,-0.110933,-0.075909,-0.111641,-1.159649,-1.160129,-1.373723,-1.353918,-0.295982,-1.403215,-0.975738,-0.975237,-0.168742,-0.263290,-0.186389,-0.250875,-1.015963,-1.016230,-0.968903,0.146997,1.366287,-0.464773,-1.116918,-1.116948,-0.216814,0.634272,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,-0.049707,-0.165054,-0.028741,-0.035391,-0.042955,-0.013282,-0.055298,-0.170400,-0.172217,-0.175227,0.887058,0.884557,-0.139729,-0.148911,-0.080147,-0.155660,-0.010763,-0.012107,-0.139731,-0.148906,-0.080147,-0.155660,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.129496,0.003143,0.002426,-0.122974,-0.041556,0.012549,-0.032244,-1.159649,-1.160129,-1.373902,-1.356210,-0.301548,-1.404577,-0.975738,-0.975237,-0.168742,-0.192468,-0.099790,-0.182133,-1.015963,-1.016230,-0.968903,-1.421138,-0.453330,-1.375441,-1.116918,-1.116948,-0.216814,0.407161,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,0.043598,-0.163905,1.059993,2.397457,-0.063200,-0.013282,4.800240,0.174329,-0.172908,0.619399,-0.243300,-0.243452,-0.139732,-0.146895,-0.079667,-0.155554,0.773847,0.378516,-0.139734,-0.146891,-0.079667,-0.155553,0.773932,0.378593,-0.024669,-0.031271,-0.023044,-0.026215,0.101008,...,0.019771,-0.119195,0.147881,0.956170,0.759756,-0.041556,-0.188716,0.149949,-0.047722,-0.048171,-1.373723,-1.356131,-0.301754,-1.404445,0.183133,0.182748,-0.163577,-0.456561,-0.424672,-0.436915,0.143001,0.142525,-0.425896,-1.421138,-1.059868,-1.071885,0.185597,0.185492,-0.216814,-0.617907,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,0.041399,0.935886,-0.098695,1.203038,0.988902,0.341693,-0.013282,-0.057392,0.058329,-0.032930,-0.042709,0.065455,0.324947,-0.139732,-0.148854,-0.080105,-0.155638,1.313051,1.106965,-0.139734,-0.148850,-0.080104,-0.155638,1.313130,1.106995,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.077909,-0.093120,1.587690,1.586539,-0.101820,0.704558,0.448898,0.659662,0.311091,0.302198,-1.373962,-1.355393,-0.300941,-1.404583,0.064846,-0.035232,-0.168742,-0.442156,-0.416540,-0.418273,0.367605,0.392035,-0.9689

,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
3,232438397,2
4,230460314,unknown


# Node2vec and anomaly detection

In [0]:
i= [0.25,0.5]
j= [ 0.1,0.5,1,10, 100]
for p in i:
    for q in j:
        node2vec = Node2Vec(graph, dimensions = 128, walk_length=20, 
                                        num_walks = 50, p=p, q=q, 
                                        workers=1)
        model = node2vec.fit(window=10, min_count=1, batch_words=4)
        #extracting embeddings for anomaly detection
        node2vec_dict = {}
        for node in graph.nodes():
            node = str(node)
            node2vec_dict[node] = model.wv[node]
        
        node2vec_pd = pd.DataFrame.from_dict(node2vec_dict, orient='index')
        #node2vec_pd.to_csv(r'C:\Users\AMALESH\Desktop\node2vec.csv')
        
        
        x=node2vec_pd
        print("{}{}=pd.DataFrame.from_dict(node2vec_dict, orient='index')".format(i,j))
        #anomaly detection using isolation and local outlier factor
        classifiers = {
            "Isolation Forest":IsolationForest(n_estimators=500, max_samples=len(x), 
                                               contamination=0.108,random_state=123, verbose=0,behaviour="new"),
            "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
                                                      leaf_size=30, metric='minkowski',
                                                      p=2, metric_params=None, contamination=0.108),
        }
        for i, (clf_name,clf) in enumerate(classifiers.items()):
            #Fit the data and tag outliers
            if clf_name == "Local Outlier Factor":
                y_pred = clf.fit_predict(x)
                scores_prediction = clf.negative_outlier_factor_
            else:    
                clf.fit(x)
                scores_prediction = clf.decision_function(x)
                y_pred = clf.predict(x)
            #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
            y_pred[y_pred == 1] = 0
            y_pred[y_pred == -1] = 1
            y_num= classes['class']
            #filtering the given classes and replacing the licit transactions (2) as 0 (according to the predictions of anomaly detection)
            y_num=y_num.replace('unknown', numpy.NaN)
            y_num=y_num.replace('2','0' )
        
            y_num= y_num.to_frame()
            y_num['y_pred']= y_pred
            #filter the columns of only given transaction class
            y_num=y_num.dropna()
        
            y_num['class']=pd.to_numeric(y_num['class'])
            y_num['y_pred']=pd.to_numeric(y_num['y_pred'])
            n_errors = (y_num['y_pred'] != y_num['class']).sum()
            # Run Classification Metrics
            print("node2vec parameters p:{},q:{}".format(node2vec.p,node2vec.q), file=open(r"/content/output.txt", "a"))
            print("{}: {}".format(clf_name,n_errors), file=open(r"/content/output.txt", "a"))
            print("Accuracy Score :", file=open(r"/content/output.txt", "a"))
            print(accuracy_score(y_num['class'],y_num['y_pred']), file=open(r"/content/output.txt", "a"))
            print("Classification Report :", file=open(r"/content/output.txt", "a"))
            print(classification_report(y_num['class'],y_num['y_pred']), file=open(r"/content/output.txt", "a"))
            print("confusion matrix:", file=open(r"/content/output.txt", "a"))
            print(confusion_matrix(y_num['class'],y_num['y_pred']), file=open(r"/content/output.txt", "a"))
            print("ROC_AUC:", file=open(r"/content/output.txt", "a"))
            print(roc_auc_score(y_num['class'],y_num['y_pred']), file=open(r"/content/output.txt", "a"))

exit()

Generating walks (CPU: 1): 100%|██████████| 50/50 [18:49<00:00, 23.22s/it]
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning
Generating walks (CPU: 1): 100%|██████████| 50/50 [19:02<00:00, 22.76s/it]
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


In [0]:
i= [16,32]
j= [ 4,10]
for p in i:
    for q in j:
        node2vec = Node2Vec(graph, dimensions = p, walk_length=20, 
                                        num_walks = 50, p=1, q=1, 
                                        workers=1)
        model = node2vec.fit(window=q, min_count=1, batch_words=4)
        #extracting embeddings for anomaly detection
        node2vec_dict = {}
        for node in graph.nodes():
            node = str(node)
            node2vec_dict[node] = model.wv[node]
        exec("node2vecd{}w{}=pd.DataFrame.from_dict(node2vec_dict, orient='index')".format(p,q))
exit()

import dill
dill.dump_session('node2vec.pkl')

Generating walks (CPU: 1): 100%|██████████| 50/50 [21:42<00:00, 26.19s/it]


In [0]:
node2vec = Node2Vec(graph, dimensions = 128, walk_length=20, 
                                num_walks = 50, p=0.5, q=10, 
                                workers=1)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
#extracting embeddings for anomaly detection
node2vec_dict = {}
for node in graph.nodes():
    node = str(node)
    node2vec_dict[node] = model.wv[node]

node2vec_pd = pd.DataFrame.from_dict(node2vec_dict, orient='index')
#node2vec_pd.to_csv(r'C:\Users\AMALESH\Desktop\node2vec.csv')

node2vec_pd.to_csv(r'/content/node2vec.csv')

Generating walks (CPU: 1): 100%|██████████| 50/50 [23:16<00:00, 28.19s/it]


In [0]:
!pip install graph2vec-learn
import graph2vec


  Created wheel for graph2vec-learn: filename=graph2vec_learn-0.1.3-cp36-none-any.whl size=5709 sha256=26eb8a3872b752bfef6c6012a71b11203ede155951c75ae9d21631b752dc98bc
  Stored in directory: /root/.cache/pip/wheels/0d/96/f7/beab1e19b0a3aad6e7ce16093548b44011a03108c2b9819592
Successfully built graph2vec-learn


In [0]:
node2vec = graph2vec.Node2Vec(threads=8,epochs = 5, walklen=20, 
                                 return_weight=0.5, neighbor_weight=10, 
                                w2vparams={'window': 10,
                                'min_count':1,
                                'batch_words': 4})
model = node2vec.fit(graph,verbose=1)

Making walks... Done, T=17.62
Mapping Walk Names... Done, T=38.34
Training W2V... Done, T=2022.51


In [0]:
print(model)

None


In [0]:
graph.nodes()